In [1]:
import pandas as pd

In [2]:
# Reading all excel sheets
allotment_status = pd.read_excel('python_dataset.xlsx', sheet_name='Allotment Status Report')
repair_penalty = pd.read_excel('python_dataset.xlsx', sheet_name='Repair Penalty')
dm_history = pd.read_excel('python_dataset.xlsx', sheet_name='DM List History')
dm_type = pd.read_excel('python_dataset.xlsx', sheet_name='DM Type')


In [3]:
# for checking "MH01DR7075" car_number
# allotment_status[allotment_status["Car Number"]=="MH01DR7075"]

In [4]:
# Select rows where 'Return Date' is not null
driver_records = allotment_status[~allotment_status['Return Date'].isna()]
driver_records.head()

,Car Number,Allotment Date,ETM,Return Date
2,MH01CV2171,2021-04-07,ETM05532,2022-01-03 15:19:06
4,MH01CV9197,2021-04-12,Leasing,2022-01-07 12:43:11
5,MH01DR1868,2021-04-14,ETM07507,2021-10-25 14:04:00
7,MH01DR1853,2021-04-16,ETM06194,2022-05-23 14:48:53
11,MH01DR1867,2021-04-21,ETM08098,2021-11-13 17:34:00


In [5]:
# car number repeation count for checking duplicates
# car_number_counts = repair_penalty['Car Number'].value_counts()
# car_number_counts.head()

In [6]:
# repair_penalty[repair_penalty["Car Number"]=="MH01DR7075"]

In [7]:
# Split comma-separated values into separate rows using explode keyword
repair_penalty['Panel Types'] = repair_penalty['Panel Types'].str.split(', ')
repair_penalty=repair_penalty.explode('Panel Types')
repair_penalty.head()

,Timestamp,Car Number,Panel Types
0,2022-07-06 18:45:57,MH01DR2761,Bonnet-300
1,2022-07-07 16:34:59,MH01DR5671,Front Bumper-300
2,2022-07-07 19:21:06,MH01CV7902,Front Bumper-300
3,2022-07-08 12:10:36,MH01DR7075,Front Bumper-300
3,2022-07-08 12:10:36,MH01DR7075,LHS fender-300


In [8]:
# Split the "Panel Type" column into "Panel Type" and "Amount" columns based on string and digit values
repair_penalty[['Panel Types', 'Amount']] = repair_penalty['Panel Types'].str.split('(\d+)', expand=True).iloc[:, [0, 1]].fillna('')
repair_penalty['Amount'] = repair_penalty['Amount'].str.strip()
repair_penalty['Panel Types'] = repair_penalty['Panel Types'].str.replace('-', '').str.strip()
repair_penalty.head(2)


,Timestamp,Car Number,Panel Types,Amount
0,2022-07-06 18:45:57,MH01DR2761,Bonnet,300
1,2022-07-07 16:34:59,MH01DR5671,Front Bumper,300


In [9]:
# repair_penalty[repair_penalty["Car Number"]=="MH01DR7075"]

In [10]:
# dm_history[dm_history["Car No"]=="MH01DR7075"]

In [11]:
# dm_type[dm_type['Cities'] == 'Mumbai 60:40']

In [12]:
# selected a column "DM Name" based on the cities "Mumbai 60:40" split
eligible_teams = dm_type[dm_type['Cities'] == 'Mumbai 60:40']['DM NAME']

In [13]:
eligible_teams

4     Silent Killers
5     Black Panthers
6       Deep Hunters
8    Terrific Tigers
9      Roaring Lions
Name: DM NAME, dtype: object

In [14]:
# Step 2: Match Repair Penalty records with appropriate team names
matching_records = repair_penalty.merge(dm_history, left_on='Car Number', right_on='Car No', how='outer')

# Convert timestamp to date format
matching_records['Timestamp'] = pd.to_datetime(matching_records['Timestamp']).dt.date

# selected timestamp which comes under in date and end column 
matching_records = matching_records.query('Timestamp >= Date and Timestamp <= End and DM in @eligible_teams')
matching_records = matching_records[['Timestamp', 'Car Number', 'DM']].rename(columns={'DM': 'Team Name'})


matching_records

,Timestamp,Car Number,Team Name
6,2022-07-07,MH01DR5671,Terrific Tigers
11,2022-07-08,MH01DR7075,Terrific Tigers
12,2022-07-08,MH01DR7075,Terrific Tigers
14,2022-07-08,MH01DR3899,Terrific Tigers
18,2022-07-09,MH01DR4159,Black Panthers
19,2022-07-09,MH01DR4159,Black Panthers
20,2022-07-09,MH01DR8718,Black Panthers
21,2022-07-09,MH01DR6854,Deep Hunters
22,2022-07-09,MH01DR6854,Deep Hunters
25,2022-07-12,MH01DR6643,Terrific Tigers


In [15]:
# merged all required dataframes
final_df = matching_records.merge(repair_penalty, on='Car Number', how='inner')
final_df = final_df.merge(driver_records, on='Car Number', how='left')


In [16]:
# final_df[final_df["Car Number"]=="MH01DR7075"]

In [17]:
# reordered column based on given output
final_df = final_df.rename(columns={'Timestamp_y': 'Timestamp','Team Name': 'DM Name'})
final_df = final_df[["Timestamp","Car Number","Panel Types","Amount","ETM","DM Name"]]
final_df = final_df.drop_duplicates()


In [18]:
final_df

,Timestamp,Car Number,Panel Types,Amount,ETM,DM Name
0,2022-07-07 16:34:59,MH01DR5671,Front Bumper,300,ETM01386,Terrific Tigers
1,2022-07-07 16:34:59,MH01DR5671,Front Bumper,300,ETM16956,Terrific Tigers
2,2022-07-07 16:34:59,MH01DR5671,Front Bumper,300,ETM17256,Terrific Tigers
3,2022-07-07 16:34:59,MH01DR5671,Front Bumper,300,ETM18374,Terrific Tigers
4,2022-07-07 16:34:59,MH01DR5671,Front Bumper,300,ETM18472,Terrific Tigers
...,...,...,...,...,...,...
516,2022-07-23 13:04:32,MH01CV6956,Front Bumper,300,Repairing,Terrific Tigers
517,2022-07-23 13:04:32,MH01CV6956,Front Bumper,300,ETM19639,Terrific Tigers
518,2022-07-25 13:15:12,MH01DR8896,Dicky,300,ETM22139,Black Panthers
519,2022-07-25 16:24:01,MH01DR8680,Front Bumper,300,ETM22105,Black Panthers


In [20]:
final_df.to_excel("final_output.xlsx")